In [1]:
import pyigtl
from matplotlib import pyplot as plt
import numpy as np
from time import sleep
from threading import Thread
from YOLOv5 import YOLOv5
from IPython.display import clear_output
import torch
from queue import SimpleQueue


weights = f'D:/GitRepos/aigt-LIVE/UltrasoundObjectDetection/YOLOv5/dummyRun/weights/best.pt'
target_size = 512
line_thickness = 2
model = None
host = "127.0.0.1"
input_port = 18944
output_port = 18945
confidence_threshold = 0.5
message_queue = SimpleQueue()

input_client = pyigtl.OpenIGTLinkClient(host=host, port=input_port)

while True:
    message = input_client.wait_for_message("Image_Reference", timeout=-1)
    if message:
        if model is None:
            input_size = message.image.shape[1:3]
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = YOLOv5(weights=weights,
                           device=device,
                           line_thickness=line_thickness,
                           input_size=input_size,
                           target_size=target_size)

        image = np.rot90(np.transpose(message.image, (1,2,0)), 2)

        prediction = model.predict(image, confidence_threshold)
        image_message = pyigtl.ImageMessage(np.flip(np.flip(prediction, axis=1), axis=2), device_name="Inference")
        input_client.send_message(image_message, wait=True)
        



KeyboardInterrupt: 

In [6]:
import pyigtl
from matplotlib import pyplot as plt
import numpy as np
from time import sleep
from threading import Thread
from YOLOv5 import YOLOv5
from IPython.display import clear_output
import torch
from queue import SimpleQueue
from threading import Thread


weights = 'weights/lung_us_pretrained.pt'
target_size = 512
line_thickness = 2
model = None
host = "127.0.0.1"
input_port = 18944
output_port = 18945
confidence_threshold = 0.5
input_queue = SimpleQueue()
output_queue = SimpleQueue()
input_device = "Image_Reference"
output_device = "Inference"
input_client = pyigtl.OpenIGTLinkClient(host=host, port=input_port)
output_client = pyigtl.OpenIGTLinkClient(host=host, port=output_port)


input_size = (800,600)  #message.image.shape[1:3]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLOv5(weights=weights,
                device=device,
                line_thickness=line_thickness,
                input_size=input_size,
                target_size=target_size)


def read_message():
    while True:
        message = input_client.wait_for_message(device_name=input_device, timeout=-1)
        input_queue.put(message.image)

def send_message():
    while True:
        message = pyigtl.ImageMessage(output_queue.get(), device_name=output_device)
        output_client.send_message(message, wait=True)

def process_message():
    while True:
        image = np.rot90(np.transpose(input_queue.get(), (1,2,0)), 2)
        prediction = model.predict(image, confidence_threshold)
        output_queue.put(np.flip(np.flip(prediction, axis=1), axis=2))

read_message_thread = Thread(target=read_message, name='read_message_thread')
process_message_thread = Thread(target=process_message, name='process_message_thread')
send_message_thread = Thread(target=send_message, name='send_message_thread')

read_message_thread.start()
process_message_thread.start()
send_message_thread.start()

Traceback (most recent call last):
  File "c:\Users\robertzsolt\anaconda3\envs\pytorch\lib\site-packages\pyigtl\comm.py", line 332, in _client_thread_function
    while self._receive_message_from_socket(self.socket):
  File "c:\Users\robertzsolt\anaconda3\envs\pytorch\lib\site-packages\pyigtl\comm.py", line 129, in _receive_message_from_socket
    body += ssocket.recv(body_size - received_body_size)
socket.timeout: timed out
ERROR:root:Error while receiving data: timed out
Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients
